In [1]:
from bs4 import BeautifulSoup
import requests
import json
import time
import pandas as pd


In [2]:
def get_type_code(r):
    res = r.content
    soup = BeautifulSoup(res, 'html.parser')
    typeks = [[x['value'],x.string] for x in soup.select('select[name="TYPEK"] > option')]
    codes = [[x['value'],x.string] for x in soup.select('select[name="code"] > option') if x.string]
    return typeks, codes

In [3]:
def get_payload(typek, code):
    payload = {'encodeURIComponent':'1',
          'step':'1',
          'firstin':'1',
          'TYPEK':typek,
          'code':code}
    return payload

In [4]:
def get_table(s, url, payload):

    # use payload 
    response = s.post(url,data=payload)
    res = response.content
    soup = BeautifulSoup(res, 'html.parser')
    table = soup.find(id='table01')
    return table

In [5]:
def get_df(table):
    raws = table.find_all('table')[1].find_all('tr')
    # get the header 
    header = raws[0].find_all('th')
    header_of_table = [x.get_text() for x in header]
    # get the cell
    list_of_talbe=[]
    for raw in raws:
        r =  [x.get_text() for x in raw.find_all('td')]
        if len(r) > 0:
            list_of_talbe.append(r)
            
    df = pd.DataFrame(list_of_talbe,columns=header_of_table)
    return df

In [24]:
# get the session
url='http://mops.twse.com.tw/mops/web/t51sb01'
s = requests.Session()
r = s.get(url)
typeks, codes = get_type_code(r)

codes

[['01', '水泥工業'],
 ['02', '食品工業'],
 ['03', '塑膠工業'],
 ['04', '紡織纖維'],
 ['05', '電機機械'],
 ['06', '電器電纜'],
 ['21', '化學工業'],
 ['22', '生技醫療業'],
 ['07', '化學生技醫療'],
 ['08', '玻璃陶瓷'],
 ['09', '造紙工業'],
 ['10', '鋼鐵工業'],
 ['11', '橡膠工業'],
 ['12', '汽車工業'],
 ['24', '半導體業'],
 ['25', '電腦及週邊設備業'],
 ['26', '光電業'],
 ['27', '通信網路業'],
 ['28', '電子零組件業'],
 ['29', '電子通路業'],
 ['30', '資訊服務業'],
 ['31', '其他電子業'],
 ['13', '電子工業'],
 ['23', '油電燃氣業'],
 ['14', '建材營造'],
 ['15', '航運業'],
 ['16', '觀光事業'],
 ['17', '金融保險業'],
 ['18', '貿易百貨'],
 ['19', '綜合企業'],
 ['20', '其他'],
 ['91', '存託憑證']]

In [34]:
#typeks = [[u'sii', u'\u4e0a\u5e02']]
#codes = [[u'01', u'\u6c34\u6ce5\u5de5\u696d']]

#if r.ok:
#        for code in codes: 
#            try:

#Jay - \0 can list all kind of listed shares. 01-水泥/02-食品/03-塑膠
payload = get_payload(typek[0], '\0')
print( payload )
table = get_table(s, url, payload)
df = get_df(table)
df.to_excel('All_Listed_Shares.xlsx')
#            except:
#                print('%s, %s faild' % (typek[1], code[1]))
#            time.sleep(5)


{'encodeURIComponent': '1', 'step': '1', 'firstin': '1', 'TYPEK': 'sii', 'code': '\x00'}
